# <i> Multiple Pooling Convolutional Neural Network 

In [12]:
from tensorflow import keras
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf 

class Cnn_Model():
    def __init__(self, size, channels):
        self.size = size
        self.channels = channels
        self.input_ = None
        self.model = None
        self.input_shape = (self.size, self.size, self.channels)

    def build(self):
        self.input_ = keras.layers.Input(self.input_shape)

        # first block
        conv1 = keras.layers.Conv2D(
            32, (3, 3), activation='relu', padding='same')(self.input_)
        
        # multiple pooling part  ------> 
        
        # pooling on the right side 
        pool_max = keras.layers.MaxPooling2D(pool_size=(2, 2))(conv1)
        
        # getting pooling on the left side 
        nega = tf.math.negative(conv1) 
        nega_pool = keras.layers.MaxPooling2D(pool_size = (2,2))(nega) 
        # taking negativ again 
        pool_range = tf.math.negative(nega_pool) 
        # gettting the range pooling 
        range_pool = tf.math.subtract(pool_max,pool_range)
        
        # concatenating pools 
        concatenated = tf.keras.layers.Concatenate()([pool_max, range_pool]) 
        
        # -----------> 
        
        
        norm1 = keras.layers.BatchNormalization(axis=-1)(concatenated)
        drop1 = keras.layers.Dropout(rate=0.2)(norm1)  # adding regularization


        # flattening weights
        flatten = keras.layers.Flatten()(drop1)

        # fully connected set 1
        hidden1 = keras.layers.Dense(512, activation='relu')(flatten)
        norm3 = keras.layers.BatchNormalization(axis=-1)(hidden1)
        drop3 = keras.layers.Dropout(rate=0.2)(norm3)

        # fully connected set2
        hidden2 = keras.layers.Dense(512, activation='relu')(drop3)
        norm4 = keras.layers.BatchNormalization(axis=-1)(hidden2)
        drop4 = keras.layers.Dropout(rate=0.2)(norm4)

        # output layer
        self.output_ = keras.layers.Dense(4, activation='softmax')(drop4)

    def compile(self):
        # model compiling
        self.model = keras.Model(inputs=self.input_, outputs=self.output_)
        self.model.compile(optimizer='adam',
                           loss='categorical_crossentropy',
                           metrics=['accuracy'])

    def fit_model(self, X_train, y_train, batch_size, verbose, epochs, validation_split, shuffle):
        history = self.model.fit(np.array(X_train),
                                 y_train,
                                 batch_size=batch_size,
                                 verbose=verbose,
                                 epochs=epochs,
                                 validation_split=validation_split,
                                 shuffle=shuffle)
        return history

    def get_summary(self):
        return self.model.summary()

In [13]:
model = Cnn_Model(224,3)

In [14]:
model.build()

In [15]:
model.compile()

In [16]:
model.get_summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv2d_3 (Conv2D)               (None, 224, 224, 32) 896         input_4[0][0]                    
__________________________________________________________________________________________________
tf.math.negative_4 (TFOpLambda) (None, 224, 224, 32) 0           conv2d_3[0][0]                   
__________________________________________________________________________________________________
max_pooling2d_6 (MaxPooling2D)  (None, 112, 112, 32) 0           tf.math.negative_4[0][0]         
______________________________________________________________________________________________